In [1]:
import glob
import os
import pandas as pd
import numpy as np 
from fancyimpute import IterativeImputer as MICE

# Data Loading

In [2]:
#laod all different user files related to the recoreded appevents into one dataframe
path_appevents = 'Data for thesis students/data_myphone/appevents'
csv_files_appevents = glob.glob(os.path.join(path_appevents, "*.csv"))

df_appevents = pd.DataFrame()
for f in csv_files_appevents:
    csv = pd.read_csv(f, sep = ";")
    df_appevents = pd.concat([df_appevents, csv])
    
df_appevents = df_appevents.drop(df_appevents.columns[0], axis = 1)

In [3]:
#load the dataframe with the categories and app names
df_playstore = pd.read_csv('cleaned_data/playstore.csv')
df_playstore = df_playstore.drop(df_playstore.columns[0], axis = 1)

In [4]:
#load demographics dataframe and keys for merging
df_demographics = pd.read_csv('cleaned_data/demographics.csv')
df_demographics = df_demographics.drop(df_demographics.columns[0], axis = 1)
df_key = pd.read_csv('Data for thesis students/anonymized_key.csv')
df_key = df_key.drop(df_key.columns[0], axis = 1)

alternative for categories
df_categories = pd.read_csv('cleaned_data/categories_short.csv')
df_categories = df_categories.drop(df_categories.columns[0], axis = 1)

In [5]:
df_appevents.columns

Index(['endTimeMillis', 'id', 'model', 'session', 'startTimeMillis',
       'startTime', 'endTime', 'notification', 'notificationId', 'application',
       'battery', 'latitude', 'longitude', 'studyKey', 'surveyId',
       'data_version'],
      dtype='object')

In [7]:
df_playstore.columns

Index(['App Id', 'App Name', 'Category'], dtype='object')

In [6]:
df_appevents.head()

,endTimeMillis,id,model,session,startTimeMillis,startTime,endTime,notification,notificationId,application,battery,latitude,longitude,studyKey,surveyId,data_version
0,1.580733e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580733e+12,2020-02-03T13:26:36.106,2020-02-03T13:26:48.545,False,0.0,com.whatsapp,73.0,51.5631,5.044949,MYPHONE,09071997qujaje,1.6
1,1.580733e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580733e+12,2020-02-03T13:29:58.420,2020-02-03T13:30:57.304,False,0.0,com.instagram.android,73.0,51.5631,5.044949,MYPHONE,09071997qujaje,1.6
2,1.580733e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580733e+12,2020-02-03T13:30:58.327,2020-02-03T13:33:10.533,False,0.0,com.facebook.katana,73.0,51.5631,5.044949,MYPHONE,09071997qujaje,1.6
3,1.580735e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580735e+12,2020-02-03T13:56:12.187,2020-02-03T13:56:29.987,True,1.0,com.whatsapp,70.0,51.5631,5.044949,MYPHONE,09071997qujaje,1.6
4,1.580735e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580735e+12,2020-02-03T13:56:30.000,2020-02-03T13:56:31.461,False,0.0,com.whatsapp,70.0,51.5631,5.044949,MYPHONE,09071997qujaje,1.6


# Merging with categories and demographics

In [8]:
df_appevents = df_appevents.rename({'application': 'app_id'}, axis = 1)
df_playstore = df_playstore.rename({'App Id': 'app_id', 'Category': 'category'}, axis = 1)

In [9]:
#merge the app-logs with the corresponding app names and categories 
df = pd.merge(df_appevents, df_playstore, on = ['app_id'])

In [10]:
df.columns

Index(['endTimeMillis', 'id', 'model', 'session', 'startTimeMillis',
       'startTime', 'endTime', 'notification', 'notificationId', 'app_id',
       'battery', 'latitude', 'longitude', 'studyKey', 'surveyId',
       'data_version', 'App Name', 'category'],
      dtype='object')

In [15]:
#Merge dataframe with the keys to allow for merging with the demographic data
df = df.rename(columns={'id': 'MobileDNA'})
df = df.merge(df_key[['MobileDNA', 'Ethica']], on='MobileDNA').rename(columns={'Ethica': 'EthicaID'})
df = pd.merge(df, df_demographics, on = ['EthicaID'])
df = df.rename({'Age': 'age', 'Sex': 'sex'}, axis = 1)

In [16]:
df.head()

,endTimeMillis,MobileDNA,model,session,startTimeMillis,startTime,endTime,notification,notificationId,app_id,...,latitude,longitude,studyKey,surveyId,data_version,App Name,category,EthicaID,age,sex
0,1.580733e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580733e+12,2020-02-03T13:26:36.106,2020-02-03T13:26:48.545,False,0.0,com.whatsapp,...,51.563100,5.044949,MYPHONE,09071997qujaje,1.6,WhatsApp Messenger,Communication,User #17984,22.5,2
1,1.580735e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580735e+12,2020-02-03T13:56:12.187,2020-02-03T13:56:29.987,True,1.0,com.whatsapp,...,51.563100,5.044949,MYPHONE,09071997qujaje,1.6,WhatsApp Messenger,Communication,User #17984,22.5,2
2,1.580735e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580733e+09,1.580735e+12,2020-02-03T13:56:30.000,2020-02-03T13:56:31.461,False,0.0,com.whatsapp,...,51.563100,5.044949,MYPHONE,09071997qujaje,1.6,WhatsApp Messenger,Communication,User #17984,22.5,2
3,1.580736e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580736e+09,1.580736e+12,2020-02-03T14:20:28.245,2020-02-03T14:20:29.704,False,0.0,com.whatsapp,...,51.562456,5.048277,MYPHONE,09071997qujaje,1.6,WhatsApp Messenger,Communication,User #17984,22.5,2
4,1.580736e+12,a4e7d00a-7e96-47b6-a865-dc32f3ebc533,moto g(8) plus,1.580736e+09,1.580736e+12,2020-02-03T14:24:24.436,2020-02-03T14:25:01.988,True,1.0,com.whatsapp,...,51.562456,5.048277,MYPHONE,09071997qujaje,1.6,WhatsApp Messenger,Communication,User #17984,22.5,2


In [17]:
df.columns

Index(['endTimeMillis', 'MobileDNA', 'model', 'session', 'startTimeMillis',
       'startTime', 'endTime', 'notification', 'notificationId', 'app_id',
       'battery', 'latitude', 'longitude', 'studyKey', 'surveyId',
       'data_version', 'App Name', 'category', 'EthicaID', 'age', 'sex'],
      dtype='object')

# Data Cleaning

In [18]:
#drop duplicates
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(before - after, 'duplicate records were dropped')

#search for missing values
df.isna().sum() #age has missing value, will be imputed later 

#check the datatypes of all columns 

107122 duplicate records were dropped


endTimeMillis          0
MobileDNA              0
model                  0
session                0
startTimeMillis        0
startTime              0
endTime                0
notification           0
notificationId         0
app_id                 0
battery                0
latitude               0
longitude              0
studyKey               0
surveyId               0
data_version           0
App Name               0
category               0
EthicaID               0
age                52606
sex                    0
dtype: int64

In [20]:
#make location values which are not useful missing
df['latitude'] = df['latitude'].replace(0 ,np.NaN)
df['longitude'] = df['longitude'].replace(0 ,np.NaN)
df.isna().sum()

In [22]:
#drop users with less than 5000 recorded events 
count = df.groupby('EthicaID')['App Name'].count().reset_index()
users = count.loc[count['App Name'] < 5000]['EthicaID']
for i in users: 
    df = df[~df.EthicaID.str.contains(i)]
len(df['EthicaID'].unique()) #186 users left 

186

In [23]:
df.to_csv('cleaned_data/full_cleaned_data.csv')